In [1]:

import sympy as sp
from IPython.display import display
import random

import symqups
from symqups import s as CahillGlauberS
from symqups import *
from symqups._internal.multiprocessing import *
from symqups.objects.scalars import *
from symqups.objects.operators import *
from symqups.manipulations import *
from symqups.ordering import *
from symqups.quantization import *
from symqups.cg import *
from symqups.eom import *
from symqups.objects import *
from symqups._internal.cache import *
from symqups.star_product import *
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject
from symqups.utils import *
from typing import Sequence

# a = [alpha(i) for i in range(10)]
# aop = [annihilateOp(i) for i in range(10)]
# ad = [alphaD(i) for i in range(10)]
# adop = [createOp(i) for i in range(10)]
# x = sp.Symbol("x")
# qq = [q(i) for i in range(10)]
# pp = [p(i) for i in range(10)]
# CahillGlauberS.val = sp.Rational(1/2)
# # symqups.hbar.val = sp.Number(1)
# CahillGlauberS
a = alpha()
aop = annihilateOp()
ad = alphaD()
adop = createOp()
pi = pi.val
CahillGlauberS.val = sp.Symbol("s")
s = CahillGlauberS.val
hbar.val = 1

##

MP_CONFIG["enable"] = True
F = sp.Function("F")(aop, adop)

def dBopp_a(X):
    return (aop*X - (s+1)/2*Commutator(aop, X)).doit().expand()
def dBopp_ad(X):
    return (adop*X + (s-1)/2*Commutator(adop, X)).doit().expand()

In [ ]:
correction = {
(1,0): 0,
(0,1): 0,

(2,0): 0,
(1,1): -(1-s)/2,
(0,2): 0,

(3,0): 0,
(2,1): -(1-s)*adop,
(1,2): -(1-s)*aop,
(0,3): 0,

(4,0): 0,
(3,1): -(3/2-3*s/2)*adop**2,
(2,2): -2*(1/2-s/2)**2 - (2-2*s)*adop*aop,
(1,3): -(3/2-3*s/2)*aop**2,
(0,4): 0,

(5,0): 0,
(4,1): 0,
(3,2): 0,
(2,3): 0,
(1,4): 0,
(0,5): 0,
}

#

m,n = 3,2

# in1 = express(sOrdering(adop**m * aop**n), 1)
# display(in1)

# expr = dStar(in1 + correction[(m,n)], 
#              rho/pi)
expr = dStar(normal_order(adop**1*aop**2), 
             express(sOrdering(adop**3*aop**1), 1))

# display(expr)

display(CGTransform(expr).simplify().expand())

\alpha_{}**3*\overline{\alpha}_{}**4